# House Prices: Advanced Regression Techniques

<p align="center">
    <img src="images/housesbanner.png">
</p>

## 1. Import necessary dependencies
-----------------------------------

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from collections import OrderedDict

import radam

## 2. Accessing and Reading Data Sets
------------------------

For convenience, we already downloaded the data and stored it in the`data` directory.
To load the two CSV (Comma Separated Values) files containing training and test data respectively we use Pandas.

In [ ]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

In [ ]:
# Let’s take a look at the first 4 and last 2 features as well as the label (SalePrice) from the first 4 examples:
train_data.iloc[0:4, [0,1,2,3,-3,-2,-1]]

In [ ]:
# We can see that in each example, the first feature is the ID.
# This helps the model identify each training example. While this is convenient, it doesn't carry any information for prediction purposes.
# Hence we remove it from the dataset before feeding the data into the network.
all_features = pd.concat((train_data.iloc[:,1:-1], test_data.iloc[:,1:]))
print(all_features.shape)

## 3. Data Preprocessing
------------------------

Before we feed it into a deep network, we need toperform some amount of processing.
Let’s start with the numerical features. We begin by replacing missing values with the mean.
This is a reasonable strategy if features are missing at random.
To adjust them to acommon scale, we rescale them to zero mean and unit variance.

$$ x \leftarrow \frac{x - \mu}{\sigma} $$

In [ ]:
# The reason for ‘normalizing’ the data is that it brings all features to the same order of magnitude
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x-x.mean())/(x.std()))

# After standardizing the data all means vanish, hence we can set missing values to 0
all_features[numeric_features] = all_features[numeric_features].fillna(0)

Next we deal with discrete values. This includes variables such as 'MSZoning'. We replace them by a one-hot encoding in the same manner as how we transformed multiclass classification data into a vector of $0$ and $1$. For instance, 'MSZoning' assumes the values 'RL' and 'RM'. They map into vectors $(1,0)$ and $(0,1)$ respectively. Pandas does this automatically for us.

In [ ]:
# Dummy_na=True refers to a missing value being a legal eigenvalue, and
# creates an indicative feature for it
all_features = pd.get_dummies(all_features, dummy_na=True)
all_features.shape

You can see that this conversion increases the number of features from 79 to 331.

Finally, via the `from_numpy` attribute, we can extract the NumPy format from the Pandas dataframe and convert it into PyTorch’s native Tensor representation for training.

In [ ]:
num_train = train_data.shape[0]
num_features = all_features.shape[1]
train_features = torch.from_numpy(all_features[:num_train].values).float()
test_features = torch.from_numpy(all_features[num_train:].values).float()
train_labels = torch.from_numpy(train_data.SalePrice.values).float().view(-1, 1)

print(f'num_features: {num_features}')
print(f'num_train: {num_train}')
print(f'train_features: {train_features.size()}')
print(f'train_labels: {train_labels.size()}')
print(f'test_features: {test_features.size()}')

In [ ]:
class HousePriceDataset(Dataset):
    
    def __init__(self, train_features, train_labels, transforms=None):
        """
        Args:
            train_features (tensor): PyTorch Tensor with shape of (num_examples, features).
            train_labels (tensor): PyTorch Tensor with shape of (num_examples, 1).
            transforms (callable, optional): Optional transforms to be applied
                on a sample.
        """
        self.train_features = train_features
        self.train_labels = train_labels
        self.transforms = transforms

    def __len__(self):
        return self.train_features.size(0)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        features = self.train_features[idx]
        label = self.train_labels[idx]

        if self.transforms:
            features = self.transforms(features)

        return features, label

## 4. Training
---------------

In [ ]:
criterion = nn.MSELoss()

def get_net():
    net = nn.Sequential(
        nn.Linear(num_features, 1)
    )
    return net

With house prices, as with stock prices, we care about relative quantities more than absolute quantities.
More concretely, we tend to care more about the relative error $\frac{y - \hat{y}}{y}$ than about the absolute error $y - \hat{y}$.
For instance, if our prediction is off by USD 100,000 when estimating the price of a house in Rural Ohio, where the value of a typical house is 125,000 USD, then we are probably doing a horrible job.
On the other hand, if we err by this amount in Los Altos Hills, California, this might represent a stunningly accurate prediction (their, the median house price exceeds 4 million USD).

One way to address this problem is to measure the discrepancy in the logarithm of the price estimates. In fact, this is also the official error metric used by the compeitition to measure the quality of submissions. After all, a small value $\delta$ of $\log y - \log \hat{y}$ translates into $e^{-\delta} \leq \frac{\hat{y}}{y} \leq e^\delta$. This leads to the following loss function:
$$L = \sqrt{\frac{1}{n}\sum_{i=1}^n\left(\log y_i -\log \hat{y}_i\right)^2}$$

In [ ]:
def log_rmse(net, features, labels):
    # To further stabilize the value when the logarithm is taken, set the
    # value less than 1 as 1
    clipped_preds = torch.clamp(net(features), min=1, max=float('inf'))
    rmse = torch.sqrt(criterion(clipped_preds.log(), labels.log()))
    return rmse.item()

Unlike in previous sections, our training functions here will rely on the Adam optimizer (a slight variant on SGD that we will describe in greater detail later).
The main appeal of Adam vs vanilla SGD is that the Adam optimizer, despite doing no better (and sometimes worse) given unlimited resources for  hyperparameteroptimization, people tend to find that it is significantly less sensitive to the initial learning rate

In [ ]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()
device = torch.device('cuda:0' if train_on_gpu else 'cpu')
if not train_on_gpu:
    print('CUDA is not available. Training on CPU ...')
else:
    print('CUDA is available! Training on GPU ...')

# Model allocation
net = get_net()
net.to(device)


def train(net, train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, batch_size, writer=None):
    train_ls, test_ls = [], []
    
    trainset = HousePriceDataset(train_features, train_labels)
    trainloader = DataLoader(trainset, batch_size=batch_size,
                             shuffle=True, num_workers=4)
    
    train_features = train_features.to(device)
    train_labels = train_labels.to(device)
    test_features = test_features.to(device)
    if test_labels is not None:
        test_labels = test_labels.to(device)
    
    # The Adam optimization algorithm is used here
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
#     optimizer = radam.RAdam(net.parameters(), lr=learning_rate)
    
    # Iterate over data.
    for epoch in range(num_epochs):
#         print(f'Epoch {epoch}/{num_epochs - 1}')
#         print('-' * 10)

        for i, batch in enumerate(trainloader):
            inputs, labels = batch
            inputs = inputs.to(device)
            labels = labels.to(device)
            labels = labels.view(-1, 1)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
#             if writer is not None:
#                 writer.add_scalar('training_loss', loss,
#                                   epoch * len(trainloaders) + i)
        
        # statistics
        epoch_loss = log_rmse(net, train_features, train_labels)
        train_ls.append(epoch_loss)
#         print(f'Loss: {epoch_loss}')
        
        if test_labels is not None:
            test_ls.append(log_rmse(net, test_features, test_labels))
    
    return net, train_ls, test_ls

In [ ]:
# TensorBoard setup
# writer = SummaryWriter('runs')
EPOCHS = 100

net, train_ls, test_ls = train(net, train_features, train_labels, test_features, test_labels=None,
                               num_epochs=EPOCHS, learning_rate=0.001, batch_size=32, writer=None)

# writer.close()
plt.xlabel('Epoch Number')
plt.ylabel("Loss Magnitude")
plt.plot(list(range(EPOCHS)), train_ls)

### k-Fold Cross-Validation

When training data is scarce, we might not even be able to afford to hold out enough data to constitutea proper validation set. One popular solution to this problem is to employ *K-fold cross-validation*. Here, the original training data is split into `K non-overlapping` subsets. Then model training and validation are executed $K$ times, each time training on $K-1$ subsets and validating on a different subset (the one not used for training in that round). Finally, the training and validation error rates are estimated by averaging overthe results from the $K$ experiments.

<p align="justify">
    <img src="images/grid_search_cross_validation.png" width="50%" height="50%">
</p>

We will put this to good use to select the model design and to adjust the hyperparameters. We first need a function that returns the `i-th` fold of the data in a `k-fold` cross-validation procedure. It proceeds by slicing out the `i-th` segment as validation data and returning the rest as training data. Note that this is not the most efficient way of handling data and we would definitely do something much smarter if our dataset was considerably larger. But this added complexity might obfuscate our code unnecessarily so we can safely omit here owing to the simplicity of our problem.

In [ ]:
def get_k_fold_data(k, i, X, y):
    assert k > 1
    fold_size = X.size(0) // k
    X_train, y_train = None, None
    
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)
        X_part, y_part = X[idx], y[idx]
        if j == i:
            X_val, y_val = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = torch.cat((X_train, X_part), dim=0)
            y_train = torch.cat((y_train, y_part), dim=0)
    
    return X_train, y_train, X_val, y_val

The training and verification error averages are returned when we train $k$ times in the k-fold cross-validation.

In [ ]:
def cross_val_score(net, k, X_train, y_train, num_epochs,
                    learning_rate, batch_size):
    train_l_sum, valid_l_sum = 0, 0
    
    for i in range(k):
        data = get_k_fold_data(k, i, X_train, y_train)
        net = get_net()
        net.to(device)
        net, train_loss, val_loss = train(net, *data, num_epochs, learning_rate, batch_size)
        train_l_sum += train_loss[-1]
        valid_l_sum += val_loss[-1]
        
        if i == 0:
            plt.subplot(2, 1, 1)
            plt.plot(list(range(0, num_epochs)), train_loss)
            plt.ylabel('Training loss')
            plt.yscale('log')
            
            plt.subplot(2, 1, 2)
            plt.plot(list(range(0, num_epochs)), val_loss)
            plt.ylabel('Validation loss')
            plt.xlabel('epoch')
            plt.yscale('log')
            
            plt.title('k-Fold Cross-Validation')
            plt.show()
        print(f'Fold {i}, train rmse: {train_loss[-1]}, valid rmse: {val_loss[-1]}')
    
    return train_l_sum / k, valid_l_sum / k

### Model Selection

In this example, we pick an un-tuned set of hyperparameters and leave it up to the reader to improve the model. Finding a good choice can take quite some time, depending on how many things one wants to optimize over. Within reason, the k-fold cross-validation approach is resilient against multiple testing. However, if we were to try out an unreasonably large number of options it might fail since we might just get lucky on the validation split with a particular set of hyperparameters.

In [ ]:
K = 5
EPOCHS = 20
LEARNING_RATE = 0.001
BATCH_SIZE = 32

train_loss, val_loss = cross_val_score(net, K, train_features, train_labels, EPOCHS, LEARNING_RATE, BATCH_SIZE)
print(f'{K}-fold validation: avg train rmse: {train_loss}, avg valid rmse: {val_loss}')

## Summary
----------

- Real data often contains a mix of different datatypes and needs to be preprocessed.
- Rescaling real-valued data to zero mean and unit variance is a good default. So is replacing missing values with their mean.
- Transforming categorical variables into indicator variables allows us to treat them like vectors.
- We can use k-fold cross validation to select the model and adjust the hyper-parameters.
- Logarithms are useful for relative loss.

## Exercises
------------

1. Submit your predictions for this tutorial to Kaggle. How good are your predictions?
2. Can you improve your model by minimizing the log-price directly? What happens if you try to predictthe log price rather than the price?
3. Is it always a good idea to replace missing values by their mean? Hint - can you construct a situationwhere the values are not missing at random?
4. Find a better representation to deal with missing values. Hint - What happens if you add an indicatorvariable?
5. Improve the score on Kaggle by tuning the hyperparameters through k-fold cross-validation.
6. Improve the score by improving the model (layers, regularization, dropout).
7. What happens if we do not standardize the continuous numerical features like we have done in thissection?